In [13]:
import cv2
import numpy as np

In [14]:
def calculateSpatialInfo(img: np.ndarray) -> float:
    """
    Calculate spatial information from a grayscale image using Sobel filters.

    Args:
        img: A grayscale image as a numpy array.

    Returns:
        The spatial information as a float.
    """
    sh = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=1)
    sv = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=1)

    sobel_image = np.sqrt(np.square(sh) + np.square(sv))

    si_mean = np.sum(sobel_image) / (sobel_image.shape[0] * sobel_image.shape[1])
    si_rms = np.sqrt(np.sum(sobel_image ** 2) / (sobel_image.shape[0] * sobel_image.shape[1]))
    si_stdev = np.sqrt(np.sum(sobel_image ** 2 - si_mean ** 2) / (sobel_image.shape[0] * sobel_image.shape[1]))

    return si_stdev

In [15]:
a = cv2.imread('images/a.jpg',cv2.IMREAD_GRAYSCALE)
b = cv2.imread('images/b.jpg',cv2.IMREAD_GRAYSCALE)
c = cv2.imread('images/c.jpg',cv2.IMREAD_GRAYSCALE)
print(f'Spatial Info of A: {calculateSpatialInfo(a)}')
print(f'Spatial Info of B: {calculateSpatialInfo(b)}')
print(f'Spatial Info of C: {calculateSpatialInfo(c)}')

Spatial Info of A: 29.988785893444604
Spatial Info of B: 13.580894199461785
Spatial Info of C: 5.99936275044091
